In [10]:
# Load packages 

import pandas as pd
import numpy as np
import re
import math
import datetime
import os 

In [19]:
# Define the inputs 

exp_str = "DevSeries22"
date_match_out = "220214"

In [20]:
# Use packages and inputs to define file names 

today = datetime.date.today()

input_folder = exp_str + "_Phos_Matching_Output" 
input_file = date_match_out + "_" + exp_str + "_Matched_Filtered0p75.csv"
input_path = os.path.join(input_folder, input_file)

output_file = today.strftime("%y%m%d") + "_" + exp_str + "_Matched_Filtered0p75_LTinfo.csv"
output_path = os.path.join(input_folder, output_file)

In [21]:
# Load and look at the human info database

df_HumanInfo = pd.read_csv("Phosphorylation_site_dataset_032020",sep='\t', skiprows = (0,1))
df_HumanInfo.head(10)

,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,LT_LIT,MS_LIT,MS_CST,CST_CAT#
0,1110035H17Rik,1110035H17Rik,Q9CTA4,7|7,S10-p,7231581,mouse,24.31,NaN,RPPPGSRstVAQSPP,NaN,1.0,NaN,NaN
1,1110035H17Rik,1110035H17Rik,Q9CTA4,7|7,T11-p,7231583,mouse,24.31,NaN,PPPGSRstVAQSPPQ,NaN,1.0,NaN,NaN
2,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,______MtMDksELV,NaN,3.0,1.0,NaN
3,Ywhab,14-3-3 beta,Q9CQV8,2|2 H3,T2-p,15718712,mouse,28.09,NaN,______MtMDksELV,NaN,2.0,NaN,NaN
4,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,__MtMDksELVQkAk,NaN,8.0,NaN,NaN
5,Ywhab,14-3-3 beta,Q9CQV8,2|2 H3,S6-p,15718709,mouse,28.09,NaN,__MtMDksELVQkAk,NaN,2.0,NaN,NaN
6,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,LAEQAERyDDMAAAM,NaN,NaN,4.0,NaN
7,Ywhab,14-3-3 beta,P35213,3q42,Y21-p,3426383,rat,28.05,14-3-3,LAEQAERyDDMAAAM,NaN,NaN,5.0,NaN
8,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,AAAMkAVtEQGHELs,NaN,NaN,1.0,NaN
9,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,tEQGHELsNEERNLL,NaN,4.0,NaN,NaN


In [22]:
# get just the residue number from "MOD_RSD"

to_regexp = df_HumanInfo.MOD_RSD
to_regexp.to_numpy()
m = []
out_res_num = []
for i in range(len(to_regexp)):
    m = re.search('(?<=[T|S|Y])\w+',to_regexp[i])
    
    if (m is not None):
        out_res_num.append(m.group(0)) 
    
    else:
        out_res_num.append(math.nan)
        
df_HumanInfo["RES_NUMBER"] = np.array(out_res_num)

In [23]:
df_XenHumanMatches = pd.read_csv(input_path)
df_XenHumanMatches.head()

,Xenopus_Reference,Human_Reference,Match_Code,Xen_Residue,Human_Residue,Xen_Motif,Human_Motif,Motif_Score
0,gnl|gene10020|XL-9_2-gene10020|rna29121|,GN:PHAX|RNUXA|human|Q9H814,0,378,349,FQDHDDTSRETFA,FQEDDDTSRETFA,0.85
1,gnl|gene10088|XL-9_2-gene10088|rna29330|,GN:FRMD8|FRMD8|human|Q9BZ67,0,410,419,RMKHLTTIDYVED,RIQHLSTIDYVED,0.85
2,gnl|gene10088|XL-9_2-gene10088|rna29330|,GN:FRMD8|FRMD8|human|Q9BZ67,0,428,437,RVKPKRTASFFTR,RVKPKRTTSFFSR,0.89
3,gnl|gene10088|XL-9_2-gene10088|rna29330|,GN:FRMD8|FRMD8|human|Q9BZ67,0,430,439,KPKRTASFFTRQN,KPKRTTSFFSRQL,0.78
4,gnl|gene100|XL-9_2-gene100|rna281|,GN:DHX16|DDX16|human|O60231,0,191,205,RSDKKAYEEAQKR,RSDKKAYEEAQKR,1.00


In [24]:
# get the ACC_ID part of the human reference 
ref_to_split = df_XenHumanMatches.Human_Reference
ref_to_split.to_numpy()
out_split = []
for i in range(len(ref_to_split)): out_split.append(ref_to_split[i].split('|')[3])
df_XenHumanMatches["Human_ACCID"] = np.array(out_split)

In [25]:
# now we want to cross reference between the human matches and the information from the human database

# store in the indexes for the cross references into human 
save_ind_to_HumanInfo = np.empty(df_XenHumanMatches.shape[0],)

save_LT_info = np.empty([df_XenHumanMatches.shape[0],1])
save_AB_info = np.empty([df_XenHumanMatches.shape[0],1],dtype='S20')

for i in range(df_XenHumanMatches.shape[0]): 
    
    #out_ACC_ID = np.where(df_HumanInfo.ACC_ID == df_XenHumanMatches.Human_ACCID[i])
    out_ACC_ID = np.where(df_HumanInfo.ACC_ID == out_split[i])
    out_res =  np.where(df_HumanInfo.RES_NUMBER[out_ACC_ID[0]] == str(df_XenHumanMatches.Human_Residue[i]))
    
    if len(out_res[0]) != 0:
        #print(out_ACC_ID[0][out_res])
        
        #save_ind_to_HumanInfo[i] = out_res[0]
        #save_LT_info[i,0] = df_HumanInfo.LT_LIT[out_res[0]]
        #save_LT_info[i,1] = df_HumanInfo["CST_CAT#"][out_res[0]]
        
        save_ind_to_HumanInfo[i] = out_ACC_ID[0][out_res][0]
        save_LT_info[i,0] = df_HumanInfo.LT_LIT[out_ACC_ID[0][out_res][0]]
        #save_AB_info[i,0] = df_HumanInfo["CST_CAT#"][out_ACC_ID[0][out_res][0]]
        
    
    else:
        save_ind_to_HumanInfo[i] = math.nan
        save_LT_info[i] = math.nan

df_XenHumanMatches["Human_LT_Info"] = save_LT_info[:,0]
#df_XenHumanMatches["Human_CST_#s"] = save_AB_info[:,0]

df_XenHumanMatches.head()

,Xenopus_Reference,Human_Reference,Match_Code,Xen_Residue,Human_Residue,Xen_Motif,Human_Motif,Motif_Score,Human_ACCID,Human_LT_Info
0,gnl|gene10020|XL-9_2-gene10020|rna29121|,GN:PHAX|RNUXA|human|Q9H814,0,378,349,FQDHDDTSRETFA,FQEDDDTSRETFA,0.85,Q9H814,NaN
1,gnl|gene10088|XL-9_2-gene10088|rna29330|,GN:FRMD8|FRMD8|human|Q9BZ67,0,410,419,RMKHLTTIDYVED,RIQHLSTIDYVED,0.85,Q9BZ67,NaN
2,gnl|gene10088|XL-9_2-gene10088|rna29330|,GN:FRMD8|FRMD8|human|Q9BZ67,0,428,437,RVKPKRTASFFTR,RVKPKRTTSFFSR,0.89,Q9BZ67,NaN
3,gnl|gene10088|XL-9_2-gene10088|rna29330|,GN:FRMD8|FRMD8|human|Q9BZ67,0,430,439,KPKRTASFFTRQN,KPKRTTSFFSRQL,0.78,Q9BZ67,NaN
4,gnl|gene100|XL-9_2-gene100|rna281|,GN:DHX16|DDX16|human|O60231,0,191,205,RSDKKAYEEAQKR,RSDKKAYEEAQKR,1.00,O60231,NaN


In [26]:
num_LT_info = (len(save_LT_info[:,0]) - sum(np.isnan(save_LT_info[:,0])))
perc_LT = 100*np.round_(num_LT_info/len(save_LT_info), decimals=3)
print("{}% of matched residues have associated low throughput infomation.".format(perc_LT))
df_XenHumanMatches.to_csv(output_path, index = False)
print(output_path)

12.9% of matched residues have associated low throughput infomation.
DevSeries22_Phos_Matching_Output/220214_DevSeries22_Matched_Filtered0p75_LTinfo.csv
